- github:
    - https://github.com/Huanshere/VideoLingo
- 官方文档：
    - https://videolingo.io/docs/introduction
- https://colab.research.google.com/github/Huanshere/VideoLingo/blob/main/VideoLingo_colab.ipynb
- requirements
    - WhisperX（本地 whisper）

- install
    ```
    git clone https://github.com/Huanshere/VideoLingo.git
    cd VideoLingo
    
    conda create -n video python=3.12
    conda activate video
    
    # 注意这里的安装是交互式，可能需要一些手动输入的参与（或者获取 sudo 权限）
    python install 1
    
    streamlit run st.py
    ```
- docker

```
docker pull rqlove/videolingo:latest
docker run -d -p 8501:8501 --gpus all rqlove/videolingo:latest
```

### UVR

Ultimate Vocal Remover (UVR) 工具通常可以用于以下任务：

- 去除人声：从歌曲中移除人声，保留伴奏（俗称“卡拉OK”处理）。
- 提取人声：从混音中分离并提取出人声，保留干净的人声音轨。

raw_full_audio.wav
- original_vocal.wav
- background.wav

### ASR

- asr
    - chinese: BELLE-2/Belle-whisper-large-v3-zh-punct
        - Belle-whisper-large-v3-zh + punctuation mark 
    - english: large-v3

### whisperx

### transformers

```
import torch
from transformers import pipeline

transcriber = pipeline(
  "automatic-speech-recognition", 
  model="BELLE-2/Belle-whisper-large-v3-zh-punct",
  device="cuda" if torch.cuda.is_available() else "cpu"
)

transcriber.model.config.forced_decoder_ids = (
  transcriber.tokenizer.get_decoder_prompt_ids(
    language="zh", 
    task="transcribe"
  )
)

transcription = transcriber("my_audio.wav")
```